This tests the accuracy of a differential operator on a polynomial of the given order. If the operator is working to said order, then the `Actual` and `Theoretical` outputs should match (except possibly at the edges.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
# Grid parameters.
nh_min = 8
refRatio = 2
cfInter = int(nh_min / refRatio)

# Time integration parameters.
deriv = 'U'
RK = 4
order = 1

This cell contains the relevant switch information.

In [3]:
TimeIntFunc, DiffFunc = TT.SolverSwitch(deriv, RK)

This cell instantiates the Grid and the Physics objects.

In [4]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(cfInter))
omega.AddPatch(refRatio, finehalf)

This cell runs a test on the derivative operator on a polynomial which matches the given order.

In [5]:
TT.DerivPolyTest(omega, DiffFunc, order)

[[ 10.66666667   0.           0.           0.           0.           0.           0.           0.           0.           0.           0.         -10.66666667]
 [-16.          16.           0.           0.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.         -16.          16.           0.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.         -16.          16.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.         -16.          16.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.         -16.          16.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0. 